In [41]:
import whisper
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import spacy
import neuralcoref
import math
import wikipedia
import torch

from pyvis.network import Network
from GoogleNews import GoogleNews
from newspaper import Article, ArticleException
import pickle

In [42]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r, article_title, article_publish_date):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def get_textual_representation(self):
        res = ""
        res += "### Entities\n"
        for e in self.entities.items():
            # shorten summary
            e_temp = (e[0], {k:(v[:100] + "..." if k == "summary" else v) for k,v in e[1].items()})
            res += f"- {e_temp}\n"
        res += "\n"
        res += "### Relations\n"
        for r in self.relations:
            res += f"- {r}\n"
        res += "\n"
        res += "### Sources\n"
        for s in self.sources.items():
            res += f"- {s}\n"
        return res

In [3]:
model = whisper.load_model("small")

In [ ]:
result = model.transcribe("COVENOM-19.mp4")
print(result["text"])

In [4]:
extracted_text = """
Ladies and gentlemen, the world has to overcome not only the damage done to our economies and our societies by COVID-19, it also has to confront the repercussions of a dangerous clash between major global powers. History is truly at a turning point. Get back in the quest to understand coronavirus after a researcher is shot and killed. I have decided that the United States of America will renounce the use of any form of deadly biological weapons. Get your shots, get your boosters. Doctors baffled by increase in sudden adult death syndrome. We destroyed all of our seed stocks. We destroyed all of our production material at Pinebuff, Arkansas, and we completely got out of the biological warfare business. Subject today concerns CIA's involvement in the development of bacteriological warfare materials with the Army's Biological Laboratory of Fort Detrick. In the course of the investigation, CIA's laboratory storage facilities were searched and about 11 grams, a little less than half an ounce, of shellfish toxin, and eight milligrams of cobra venom were discovered in a little used vaulted storeroom in an agency building. I emailed the FBI June of 2021. I've been working on understanding what COVID was and that even though this thing has been labeled a respiratory disease and arts, we are not seeing that that's what the profile of the disease is. We are seeing it as an inflammation. They wrote back to him? Yeah, they wrote back a letter that said, received. Period. You got a three-word letter back. For years, he trains FBI and CIA agents. That's his job in the prevention of mass killings worldwide. And he specializes in bio weaponry. Various dissemination devices such as a fountain pen dart launcher and an agent's head bolt designed to release when heat appeared to be peculiar to students for agents for fine dust and use. The first drug approved by the FDA to treat COVID-19 patients. What if we've just missed understanding what the real weapon has been the whole time? Why I dug and the more I looked, the more obvious it became that everyone needs to be looking at this. Everyone needs to be looking at this. I said since May 2020, remdesivir will result with at least 30% deaths of everyone who receives that drug in a hospital for five to 10 days. I got Thomas Renz to pull the CMS data. You know for the state of New York, I decided to run it in New York. I wanted to know Medicare-aged patients. What's the percentage of death of those who received remdesivir for its five-day protocol? 26.9% of them died. The reason why we're making the announcement now is something that I believe people don't fully appreciate. Whenever you have clear-cut evidence that a drug works, you have an ethical obligation to immediately let the people who are in the placebo group know so that they can have access and all of the other trials that are taking place now have a new standard of care. So you have kidney failure, liver failure, now heart failure being caused by remdesivir, published to do so. Two months later, your NIH, the FDA, still sits back and goes, this is the only approved drug for hospitalized COVID-19 patients. You know what's more disgusting? January 21st of this year, the FDA extended an emergency use authorization and said there's now only one authorized medication that can be pumped into the veins of all newborns in this country to 18-year-old pediatric age-ranged children, starting with seven-pound newborns. Guess what the only authorized drug to treat COVID-19 children is now? The FDA gave full approval to the drug remdesivir for children as young as 28 days old. And I know Brian Ardis well and I adore him and the stuff that he broke on remdesivir was life-changing for me as a doctor and his story with his father-in-law and he brought that to the entire medical world around the globe. Nobody knew that. The FDA said in a press release that it had concluded remdesivir may be effective to treat the disease and the potential benefits outweigh the risks. It has been an amazing journey to watch. Multiple people from multiple fields come together as a group to try to actually save humanity, fight against those in government and federal health agencies around the world who it appear are conspiring to actually cause harm and inflict harm on other people. As many people that would allow me to speak articulate things that brought hope and inspiration and then removed fear, that has been my number one agenda this whole time and anyone who doubts that obviously hasn't been listening. The first COVID-19 treatment for younger children now approved. The FDA now says it is safe even for babies who are just a few weeks old. It kind of just felt like the last two years I'm like this older brother trying to be this guardian of the innocent, that they're not manipulated, that they're informed decisions and informed consent are stolen from them. I wanted them to be able to have information that helped them and if it helped anyone then I felt I did my job. My work merged into working with counter-terrorism mostly as a trainer in the subject of active attackers and what people call active shooters. So I'm sort of seen in my industry as the bridge between law enforcement, public health and emergency management and then the pandemic. I just perceived that what we were going through, whether it was an attack as a virus or attack as a human being, I just saw that the role is the same. I need to understand the threat and then I needed to find mitigation strategies to either prevent the threat from having an ability to create harm or work out what the recovery looks like. So that's what I've been doing since the beginning of the pandemic. One of the things that led me to go look for venom in the first place was it didn't make sense to me biologically that a clean virus, and I call it clean because it's not very cytotoxin, the coronavirus. It's pretty clean, it's a good going in and looks for a place to basically bond. In this case with this coronavirus it's the ACE receptor and then it does its thing, it distracts the DNA and the resources and it makes more of itself. It's not necessarily going to cause that much cellular damage around it, whereas this thing is doing the opposite. This thing is really disruptive in the body, which is why I separated out what I've been sort of referring to now as the tank and the gun on top. Ten years ago I had researched something that I brought to Dr. Artis's attention just a few days ago and he was blown away because it linked a program called MKUltra that was part of the CIA's program. I'm not sure if you're familiar with it. Sure, of course. Are you familiar with the heart attack gun? No, I'm not. The woman that was tasked with it, she was only 18 at the time, her name is Mary Embry. This is all public, this was a congressional hearing. It's not me and a conspiracy website. It's a congressional hearing where they show the gun and they explain how it was developed and it was using a shellfish venom that was frozen in ice and they could shoot it about 100 meters. The CIA were using a third of all shellfish toxin available that was in existence. Arguably it was only 5.9 grams, but why would they need basically a third of the whole supply on the market if they were just trying to target a few individuals? In that they even mentioned specifically that there were 19 other toxic substances including cobra venom. Also I had to find, one time they wanted me to find out if there was such a thing as a poison that was undetectable, especially one that seemed to mimic a heart attack that would kill someone but it would appear that they had a heart attack. I did find such a thing. Was this pistol fire the dart? Yes it does, Mr. Chairman, and a special one was developed which potentially would be able to enter the target without perception. The poison was frozen into some sort of dart and then it was shot at very high speed into the person so when it reached the person it would melt inside them and the only thing would be like one little tiny red dot on their body which was hard to detect. There wouldn't be a needle left or anything like that in the person. But also the toxin itself would not appear in the autopsy? Yes, so that there was no way of perceiving that the target was him. So Frank Rudolph Emanuel Olsen was an American bacteriologist, biological warfare scientist and employee of the United States Army Biological Warfare Laboratories who worked at Camp Dietrich in Maryland which is also where they pulled the venoms from conotoxins and king cobra venoms and other things when they were doing the congressional hearing for the heart attack gun. It was from this lab? It can't be true. I was told that your father had an accident. He fell or jumped out the window and he died. He didn't show the science of impact from the fore. He showed the impacts of having been hit in the head with like a bat or something that fractured his skull but then thrown out the window and the autopsy was undeniable and this was public during the time. We didn't pass the truth serum test. He was very disturbed with what he was learning inside of this program that he was in and what he was a part of and his conscience was deeply troubled. And then you move into the 70s where the CIA is using venom as a weapon with frozen water that you just told me about yesterday. And we watched the videos of the actual congressional hearings that showed between the year 1952 and 1975 when it was disclosed this had been happening between those years and being developed and then disclosed by a gentleman Mr. Colby. It's been around for 50 years I just learned from you. I had no 50 years you don't think they've learned how to use and isolate venom peptides from animals or components of venom as a weapon. The CIA was doing it back in the 70s. What's even more incredible is that not only are they using just the shellfish toxin from the 70s as a weapon to stop the hearts of a human. And it was undetectable by science once you did an autopsy. That was incredible to hear them talk about that in the congressional hearing. That was in 75. You're now 2020 inside of these individuals all of which tested positive for COVID-19 every single tissue sample evaluated urine, feces and blood. When they evaluated all of it they not only found shellfish neurotoxin which is what the CIA was using. They actually found 20 different venom peptides that are found in venomous snakes. In 1956 there was a guy named Dr. Stanley Cohen. Dr. Stanley Cohen was the very first successful person supposedly to mix the DNA of two different species together and create a hybrid, a chimera. And he did this in the 70s. Anything that disturbed the timing of the developmental process was worthy of study. If you put in a lysosomal inhibitor such as ammonia or chloroquine it did not affect the binding but it blocked the degradation. But in 1956 he discovered that in snake venom he could actually cause neurological tumors with a factor found in venom of snakes called nerve growth factor. And then he discovered something called the epidermal growth factor which could explain tumors and other tissues of the body outside of neurological tissue caused by venoms. This company Genentech that right now has seven patented drugs for cancer treatment that are all patented components of snake venom. They're just extracts of snake venom. This company Genentech is who was bought by Gilead Sciences in 2011. Several of their labs, biological labs were purchased in San Francisco in 2011 and they brought all 55 top heads and executives into Gilead who then made Remdesivir the drug that was the one and only drug being used in all hospitals around the world right now for COVID. This section here of the document that was put out in 2012 that was put out through the Department of Justice explains that it would be unlikely for these to be actual venoms meaning straight from an animal into a person but instead it would be connected to laboratory clandestine activities and as well it would be utilizing a technology which I believe the fruition of that is peptide technology. Yeah so it says use as a terrorist weapon the conotoxins. It says conotoxins have potential as biological weapons. The direct chemical synthesis would be more likely to be found like you mentioned in clandestine laboratory than the farming of cone snails. Collecting a large enough supply directly from cone snails to use an aerosol dispersal would be a cumbersome process. Most conotoxins are small peptides with 10 to 30 amino acids which can make them relatively easy to manufacture using direct chemical synthesis as discussed above much research is being done with conotoxins. Did you know that that much research is being done with conotoxins? No. I did not. But they say the Department of Justice much research is being done with conotoxins. Had no idea. Just wanted to know why didn't the CIA destroy these toxins when the president five years earlier wrote into law that they had to be destroyed and then they were stumbling around trying to make excuses why they preserved these venomous toxins. It's very interesting. So the supplies and laboratories could be diverted to terrorists it says. So the much research being done in conotoxins the Department of Justice says here that supplies that are in laboratories supplies of conotoxins could be diverted to terrorists. The United States Department of Health and Human Services requires registration background checks bio safety and security procedures for handling alpha conotoxins at amounts exceeding 100 milligrams. The most likely method of dispersal would be as an aerosol. In theory terrorists could also use certain conotoxins to disrupt agriculture by poisoning farm animals. And this is this is amazing what I just mentioned was it's amazing this is what they found in that Italy study when they were evaluating COVID-19 patients who were positive for COVID-19 against those who did not test positive. They found 15 of these conotoxin in the COVID-19 patients. How did they get there these are marine snails how they get there. Daniel Baker thank you for interviewing with me you've been through Helen back and you're still in it. I took my first one in June. These are the signs and symptoms if you have been exposed to a cone snail venom they include faintness yes. Tosis which is drooping eyelids. Poor coordination. Absolutely. Absent gag reflex. I would imagine yes because I have troubles following sometimes. A reflexia. I'm hyper reflexia. Yeah which is still it's related. Diplopia double vision. Blurred vision. Speech difficulties. I think I'm good on that one. Absolutely I am for that. Parasthesias. Absolutely on my diagnosis list. Abnormal sensations such as burning or tingling. Urinary retention. Dysphagia. Difficulty swollen. Weakness. Yeah. Nausea. I have a constant nausea. Has numbness. Absolutely. And respiratory arrest. That's the only thing that I haven't had. It doesn't make sense that there's a neurotoxic venom from a snail in the ocean that can be used as a bio weapon now that you're learning. That I've since learned is mass manufactured in facilities since 1989 and the synthetic version of this alpha-conotoxin which is deadly, more deadly than a King Cobra's venom. The synthetic version in factories is more deadly and as biologically active as the natural version. I'd be concerned about this. In fact they just said this. They can give it to terrorists. The ease with which a biological weapons program could be disguised within legitimate activities and the rapid advances in biological research make it very difficult to detect violations. We believe that a protocol would not be able to keep pace with the rapidly changing nature of the biological weapons threat. The only downside of the study would be it's a smaller group of people, 20 people in the infected group and 10 people in the non-infected group. So what do you think about that? That is a limitation to all studies. However, there's studies with far fewer people in them that they peer review publish and use a scientific basis for any other claims all over the place. This is a very small study. You can almost call it a cohort study. My problem is it was submitted for peer review in publication June of 2020. It didn't actually get peer reviewed and published until October 2021. Why did it take so long? Why did it take a year or three months to publish this finding? What this topic highlights and I think what the bigger picture of why the public should be interested in it is like all things technology at this accelerated pace. Technology has moved so quickly now and there aren't very many regulations around things like venom. The ability to now synthesize peptides and peptides are just short chain proteins and the ability to make venom is now an easy task for people to be able to do. My name is Zoltan Takash. I'm a biomedical scientist. I follow my childhood passion. Our work is based on the genetic blueprint of the toxins, either the DNA or the RNA sequence, because we can actually, using bioinformatics, we can design the libraries. We know every single toxin variance what we create in a library which holds, let's say, 2 million toxins. We are exactly what goes into that library. And then we can put these DNA pieces artificially into bacteria and the bacteria will make as much toxins as we wish. I mean you can't connect the gun to the shooter and hold them accountable. The president currently is chasing ghost guns, right? But like ghost guns are so old school because you're thinking about gun powder, like how long has gun powder been around? And like we're not even like as human beings, like that is so primitive as a weapon. Well now if you need a piece of plastic or even metal replaced out of a machine and you had the specs on the past, you can just make one, right? You can just 3D print it. So that's where we are biologically. You can literally instruct whether it's E. coli or yeast and you can instruct them through various processes to say make this protein for me. So today about 20 different medications are derived from animal venom toxins and they're treating like heart attack, high blood pressure, heart failure, thrombosis, diabetes, chronic pain. It's actually quite interesting that how many different types of diseases could be potentially treated by animal venoms. And the reason for this is because toxins have been evolved for millions of years to immobilize and kill prey and predator. The animals such like snakes, gala monsters, centipedes and even snails are probably not what you pictured in your drug factory. But some of these venomous creatures have produced blockbuster drugs to treat diabetes, hypertension and chronic pain. So I like to describe venomics as sort of this marriage between evolution and technology. By investigating the venom of these predatory animals combined with modern day techniques like RNA and DNA sequencing, we sort of propel the discovery of novel compounds that can be used in drug development. This nail has a venom which I like to describe kind of like a cluster bomb. It's not one peptide, but it's several peptides that have evolved over time to basically shut down the function of that fish. And these are peptides that have evolved to hit their specific targets. So to give you a summary of what we've done so far, we use nature to sort of learn the lesson of which of these nails are producing venomous peptides that are manipulating signals in cells. We investigate the venomous compounds, various ones, and we try to identify the peptides for which there is a biological assay that targets cancer cells. So nature has the answers. We just have to trust in what I like to call the killer snails. Thank you very much. So let me just say again, the variants are not different enough to represent a threat to you. So you do not need to top up vaccines, yet they are being made and the regulators have more or less waved them through. I'm very frightened of that. There's no possible benign interpretation of this. I believe that they're going to be used to damage your health and possibly kill you, seriously. I can see no sensible interpretation other than a serious attempt at mass depopulation. This will provide the tools to do it and plausible deniability because they'll create another story about some sort of biological threat and you'll line up and get your top up vaccines and a few months or a year or so later you'll die of some peculiar, explicable syndrome and they won't be able to associate it with the top up vaccines. But that's my belief that they're lying to you about variants so they can make damaging top up vaccines that you don't need at all and I think they'll be used for malign purposes. And if you don't wake up, that's what's going to happen I think during next year. And I don't normally use phrases like this but I think we are standing at the very gates of hell. Okay Erik, you have sent me some videos on how you have filmed with your microscope and this was filmed last summer, 2021. He's a Swedish doctor who has for 50 years been studying under a microscope what particles and molecules in the blood look like and impact disease in patients. That's what he's done for 50 years. He spends every day looking under microscopes, looking at molecules and what he calls particles over and over and over that he has identified are biomarkers for chronic illness and disease. He actually is filming that inside the vaccine are these little organisms that are just sitting there releasing one particle after another. They are manufacturing particles. I couldn't believe what I was watching. He was describing that the particles that these little organisms inside the Pfizer vial were releasing look like the exact particles that create chronic disease and death over time. And these particles and this whatever organism is he said is an identified isn't disclosed to be in the Pfizer vaccines but he knows this is going to contribute to disease and death over time to anyone who gets these shots. The substrate he is looking at is strictly the Pfizer vaccine under the microscope. What it looks like actually now having this perspective of venom is that it looks like a snake venom gland organoid. In the Clavers lab at the Hubelast Institute we have for the first time been able to grow snake venom gland cells in the lab using organoid technology. To obtain snake venom is very important to produce anti-venom or to study snake venom from physician to use for therapeutics. So we were wondering if we can improve this process by growing many organs of the snake venom gland. For that we got eggs of various different species of snakes, we got the glands out and we isolated the stem cells and tried to grow them as many organs. First we saw that they didn't grow at 37 degrees because snakes are cold blooded so we had to lower the temperature to 32 degrees to make them grow. So this is different but was very similar and that was one of the big discoveries of the study is that a lot of the growth factors we need from mouse and for human are also required for the snake and in fact we can use the human growth factors to grow a snake venom gland. And these are actual components of snake venom glands that can be extracted and in labs can just sit there and create venom just by themselves without the snake even being attached. This has been done for decades now, they know they can use these organoids at least that I'm aware of over the last several years to just generate venom in a lab. It looks just like that in the Pfizer vaccine in Dr. Eric M.B.'s video. Venom is a very complex mixture of toxins and it was not known if different cell types contribute to different parts of the venom or if one cell can actually make everything in the venom. So using single serone sequencing in which we can look at what every cell type produces we found that different cell types actually produce different compounds and it's just not one cell that can make everything. What's great about the organoids is that every single cell is a factory for making toxins and we can expand the organoids as much as we like. We could already show that when we take toxins from our organoids and add them to muscle cells they stop firing which is similar to being paralyzed by an actual snake part. I mean who would want to spray it in here in the body? I don't want to have it. Would you be able to think about it? No, I don't want to have such an intention. The person interviewing him says I want to show you a clip of Luc Montagnier who was the guy who won the Nobel Prize for identifying HIV virus. And then when he gets done with the clip the person interviewing him looks at Dr. Eric Mbien and says do you agree with Luc Montagnier that the vaccines themselves are going to create more dangerous infectious variants? As we've been reporting the Pfizer-BioNTech vaccine must be shipped and stored at minus 70 degrees so ultra cold dry ice is now a very hot commodity. Whatever this is in the Pfizer vaccine is releasing particles as it causes them that are going to continue to cause disease and death to specific organs in the body. This isn't the vaccine causing more variants. He said what's in the shot is actually creating particles that cause disease and death of organs in the body over time. The interviewer looks at him and says what are people going to do? And he says if this is getting injected into people what he's seeing underneath the microscope in the Pfizer vaccine which in my opinion looks like an organoid that releases and creates venom he says if that's in here and it creates disease over time there are things everyone should be doing internally in their body if they're getting this injected into them that will stop this organism from releasing these particles that cause disease. And the interviewer says like what? And they talk about it. He says like vitamin C load should be high in the body because it will stop these organisms from releasing these disease creating particles. Dr. Eric NB says and they talk about vitamin D load and it speaks exactly to what we have found and uncovered when it comes to snake venom to the side effects of the chemicals that we have disclosed like polysorbate, 80, polyethylene, glycol, 2000 that's in the Pfizer and Moderna shots and Johnson and Johnson shots. We have continued to talk about it doesn't matter what goes into your body and the way of germs, pathogens, chemicals, toxins, poisons, it doesn't matter, venoms. Your internal environment is what is most important to protect you from disease from any of these toxins as you want to call it, label them. It's no different than what Eric NB says. He goes we're going to have to tell the people getting these shots. They have to keep their vitamin C load up. They got to keep their vitamin D levels up because it will restrict these organisms from being allowed or inhibit them from releasing these dangerous toxins. As this narration and discovery became more and more real to me, it became more and more obvious that the injuries from the vaccines could actually be linked to snake venom also. Imagine my shock after a few days after the snake venom watched the water documentary was released with stupefiers that there was a medical doctor out of South Africa and her name is Dr. Zondre. Dr. Zondre had been taking pictures of her patients post Pfizer vaccines for COVID-19. She was taking pictures of their blood and she was recognizing things in the blood that didn't belong there. She had never seen what she was seeing in the post vaccinated COVID-19 patients, what she was seeing in their blood. She was asking around the world for a year and a half if other people around the world knew what this was, she was seeing in the slides containing her patients blood after getting the mRNA shots and no one knew what it was. After she watched the water, she went online now having this perspective of looking for snake venom. She actually went online and this is what she typed in, what does snake venom look like in the blood of a patient under a microscope? So what I'm seeing in the blood is what they are picking up in a lot of the other blood tests as well, you know, white blood cells that are not responding and they're non-viable and then I also see these structures in the blood that is abnormal. These black crystals and obviously there's a lot of things that we have analyzed and a lot of other scientists that have come out and analyzed these structures. What she discovered on her basic search online and seeing these graphic images appear was exactly what she was looking at for the last year and a half in the blood of her patients after getting the Pfizer shots. What appears is these green crystal looking fragmented structures and this is her search and it's titled cytotoxicity or cell toxicity of snake venom, PLA2 like myotoxin which is blood or sorry muscle toxin on rat cardiomyocytes or heart cells. So this is just her own kind of Google search if you want. She just Google search what does snake venom look like in the blood of a patient under a microscope because she never looked this up before. That site is reputable. This is nature.com. This is a huge medical online journal. Now when she sent these around the world over the last year to ask what are these structures in the blood? I don't recognize these. People around the world started saying we think that might be nanotechnology. To her shock even the green hue is what it looks like underneath the microscope on her search for what snake venom looks like. That couldn't possibly be an artifact could it? You know what I'm talking about with blood analysis where say some dust gets on the microscope and so it magnifies this piece of dust. Sure. Looks like a big structure in there. Yeah. Could it be? Sure. Could be. Except it looks just like snake venom underneath the microscope. So this just got her attention that she's been seeing this consistently and there's this reaction to red blood cells when exposed to snake venom under a microscope where the red blood cells will shrink and then spike. On top of this pictures with the Pfizer vaccine she has in her clinic after our documentary dropped she actually was taking pictures of what was in the Pfizer shot just under by itself under the microscope. She goes do you know that snake venom gland organoids can just produce venom in a laboratory in a petri dish and I said yes. So you can see this little graphic of what they look like and on the screen you can see they look like, organoids look like these little bubbles. They look like air bubbles. She's like Dr. Artis this is what it looks like in the blood of a patient and this is what I'm seeing in the Pfizer vial. This actually looks a lot like what Dr. Eric Enby in Sweden said was producing little particles in the Pfizer vaccine under a microscope that he said those particles whatever is coming out of this thing in the Pfizer shot is going to cause disease death and illness of organs throughout the body. Whatever it is this thing is creating. He didn't know what it was. I actually believe it's the organoids. Which would help us understand this thing that we've been educated on which is we turn our bodies into a spike protein factor. Exactly right. The organoid explains it better than anything else. Anything else that the body is generating. I actually think spike protein basically looking at this and learning about this. It actually looks like the spike protein could be code for another five letter word. S-N-A-K-E instead of S-P-I-K-E. This is something else she learned about. Not only did she see these crystal like structures that look just like venom in the blood under microscope. She wanted to know what's the impact of snake venom on red blood cells under the microscope. What does it look like when someone gets bit by a snake? What does the venom do to red blood cells? So this is a normal red blood cell image. This is what happens after exposed to venom. And she said I saw this in almost every one of my mRNA vaccinated patients blood. And this is what she did as an experiment. When she learned online that this is the impact of snake venom on red blood cells after watching our documentary, she took her own blood and put it on a slide, looked under the microscope. She took pictures and videoed it. And you'll see all the healthy red blood cells. None of them are attached. None of them are sticking together. And then she took one drop of undiluted Pfizer vaccine. She had three undiluted Pfizer vaccines. She added one drop to her blood sample here. And within five minutes, all of her red blood cells started to actually shrink and spike exactly like she learned online is the reaction of red blood cells in humans when it's exposed to snake venom. It was very brave of her to bring this forward. But this answered for her all the mysteries and complexities of what she was seeing under the microscope and the complaints and symptoms her patients were experiencing in South Africa after being exposed to the Pfizer shots. This stacking appearance of red blood cells is a side effect of snake venom also. It hydrolyzes the phospholipids, which is the layer of the red blood cells on the outside and it makes them start sticking together. Dr. Thomas, he leave it a great job actually. He's a cardiologist out of Miami. He demonstrated on orthomolecular.org. He demonstrated that this RULU formation could be completely absolved using ultraviolet light therapy and vitamin C high dose. And there's pictures and images of how this is reversed in just minutes. This is an effect also of snake venom and of the Pfizer vaccines. I want to ask this to Carrie. How do they misuse PCR to estimate all these supposed free viral RNAs that may or may not be there? I think misuse PCR is not quite, I don't think you can misuse PCR. The results, the interpretation of it. See, if you can say, if they wanted, if they could find this virus in you at all, and with PCR, if you do it well, you can find almost anything in anybody. It starts making you believe in the sort of Buddhist notion that everything is contained in everything else. I mean, because if you can amplify one single molecule up to something that you can really measure, which PCR can do, then there's just very few molecules that you don't have at least one single one of them in your body. So that could be thought of as a misuse of it just to claim that it's meaningful. It's not an estimation, it's a really quantitative thing. It tells you something about nature and about what's there, but it allows you to take a very miniscule amount of anything and make it measurable and then talk about it in meetings and stuff like it is important. See, that's not a misuse, that's just sort of a misinterpretation. PCR is separate from that, it's just a process that's used to make a whole lot of something out of something. That's what it is. But it's not, it doesn't tell you that you're sick and it doesn't tell you that the thing you ended up with really was going to hurt you or anything like that. Changing international COVID testing rules, lacks governmental regulation and rapidly developing technologies have created confusion when it comes to PCR testing. The most accurate type of COVID test often required for worker travel. PCR tests have been a mainstay in genetic testing for snake venom. And isn't it weird that the whole world is now being forced to use a PCR test? And they're just using it and forcing everybody around the world to use it and accept it as a new reality when the inventor himself told you that's not a reality. Guys like Fauci get up there and start talking, you know, he doesn't know anything really about anything and I'd say that to his face. Nothing. The man thinks you can take a blood sample and stick it in an electron microscope and if it's got a virus in there, you'll know it. He doesn't understand electron microscopy and he doesn't understand medicine and he should not be in a position like he's in. Most of those guys up there on the top are just total administrative people and they don't know anything about what's going on. The people were hypothesizing in China and in France that these two venomous looking spike proteins that most reflected the origins from two snake venoms specifically target nicotine acetylcholine receptors in the brainstem that leads to respiratory paralysis that would look like you had a respiratory illness. And then for those who already had issues like diabetes, heart disease or previous lung issues, those with preexisting conditions, they would end up being worse off than other members of the family and end up in a hospital. There they would use hospital protocols to kill you with sedation drugs and remdesivir or in nursing homes. Load you up with bedazolamine morphine for end of life care in the UK. They would mass murder innocent people, call it they died of an infection and then try to sell you on a vaccine agenda. I'm not a nicotine expert but these researchers kept saying there's this observation, some protective mechanism of nicotine is occurring by people that use nicotine products and we're seeing it in observation with clinical settings. It's a very interesting set of circumstances. You would never think that smoking could be good for anything. But here we are with apparent evidence that suggests there's a possibility, however tenuous, that nicotine might be some kind of shield. The study from doctors at a major French hospital found everyday smokers are much less likely to develop serious COVID-19 symptoms. France then banned online sales of nicotine products ahead of an expected surge in demand. At first experts studying COVID-19 thought smoking was a major risk factor but now they're not so sure. Data collected by French scientists suggest smokers are less likely to catch the coronavirus. The experts we talked to said this is not the case. Maybe this is why we don't see smokers ending up in hospitals with COVID-19 as we do non-smokers. Maybe there's a protective benefit of nicotine. Less than 5% of all people hospitalized for COVID-19 by these months in 2020 were smokers and all of the scientists said it should have been the opposite. It should have been 95% of all hospitalized COVID-19 patients if it was a respiratory virus. It should have been smokers. But then our government decided to let everyone know around the world that it's just the opposite. They decided to tell us online that smoking is a higher risk factor for being hospitalized for COVID-19 and dying from COVID-19, which was not the observation nor was it what scientists were asking for. These studies are sponsored by companies that sell tobacco and the World Health Organization has issued a clear warning. Do not fall for such studies. Smoking or vaping could save your life. And that is especially true when it comes to the coronavirus pandemic. And another systematic review of evidence from China found the opposite. That smoking worsened COVID symptoms. If you are a smoker or a vapor, this is a very good time to stop that habit and we will help you. There is a drug that is proving to be of miraculous impact. And when I say miracle, I do not use that term lightly. Why is it potentially that these work to help somebody that has been invenomated? Yeah, so in the research study that we were talking about here in France, they actually mentioned that Ivermectin was showing promise by April of 2020 when the French study researchers noticed this. They said, we're seeing around the world that Ivermectin is having a huge benefit at helping to reduce hospitalization and death from COVID-19. Ivermectin has a second or a primary target in the central nervous system. It's called alpha seven nicotinic acetylcholine receptors. And they said, maybe this is its inhibitory effect against COVID neurotoxic effects. And maybe that's the same as what nicotine is providing also. And said, this is probably why Ivermectin is working. Well, it is true. These researchers in France actually noted that the beta version of this coronavirus, which is what SARS-CoV-2 is, is known to cross the blood-brain barrier to affect the central nervous system. Which is where these nicotinic acetylcholine receptors are found. They also noted in this study, which I found interesting, that it was being noted around the world that the spike protein on SARS-CoV-2 binds to ACE2 receptors. They make mention in this study that those proteins, though, aren't located inside the lungs. Which is interesting because if you're breathing in the spike protein tied to this coronavirus, the respiratory virus, it should be binding to ACE2 receptors in your lungs. Cerebral Pneumetroid 19 is a disease caused by a virus called SARS-CoV-2. The virus infects cells along your airways by attaching to ACE2 and other molecules on those cells. Except these researchers said, you don't find ACE2 receptors inside the lungs. But you do find it inside the brain, inside the central nervous system. Which would explain why these venom peptide toxins that looked identical to the spike protein of SARS-CoV-2, along with the rabies virus that they found inside this research study in France, why it was they were looking at the central nervous system as a primary site of target for these venom toxins, and the respiration paralysis that occurs in COVID-19 patients, and the reduction in oxygen load. It is very well known that these venoms that are neurotoxic lead to hypoxia, which is reduction in oxygen, which is exactly what every COVID-19 patient was seeing with their pulse oximeters. They were watching their blood oxygen level drop, drop, drop. If the COVID is found in wastewater, it's getting there because people in the community that have the virus are actually shedding some of it. Because not everyone who has COVID-19 is tested or has symptoms, wastewater surveillance can capture the presence of the COVID-19 virus shed by people with and without symptoms, giving a more accurate picture of how much virus is in the community. In truth, our misdeeds were few and far between, as the final Rockefeller Commission report revealed. So my team found these documents. It's the additional information from the 1975 Congressional hearing for the CIA. Page 241 of the document reads titled, Water System Test. So the General Services Administration in Fort Detrick entered into a cooperative project to investigate the vulnerability of drinking water in federal buildings to covert biological attack. Tests were actually planned and carried out in the FBI building. The second test was a simulated covert test. Neither the building occupants nor operating personnel were advised that such a test was planned. We were not challenged and apparently undetected. So clandestine, they're poisoning their own staff members, right? That's insane. Easy to get information in four categories is all the saboteur needs. Number one, the number of fountains in the building. Number two, the approximate total floor area and type of activity. Three, the approximate number of employees for the available chlorine content of water supply at time of attack. I have talked about this nonstop. My belief was they're targeting people through the water. You know the CDC, you know these federal health agencies already know the available chlorine content in the water supply to your house and to your community because they put it in there. They already have this information. They did an analysis for the available chlorine and it was positive two hours after the start of the test. The living colophage organisms were killed quickly. This is why they needed to know how much chlorine is in the water before they poison you with venomous peptides from shellfish at least in this scenario because chlorine had a obviously inhibiting effect on those organisms. This is phenomenal. They've told you the Grammage. Isn't this amazing? This is amazing. This is proving what you're talking about. This is way more plausible than it is just theory right now. This is incredible. Yeah. I couldn't be more proud of you and your team for even looking into this information to substantiate any of the claims that we're making that it could be possible. And if it is, we should be looking there. They're looking there. All I said was I don't trust the CDC what they're doing with our water and they're using that as a predictor of outbreaks in your town of a respiratory infection. I'm about to do an interview actually with a lady named Janet Fien. She wrote a book about how pandemics can be orchestrated through water delivery systems throughout all of America with a second water line. And we're going to go through her book. She's an investigative journalist looked into this for 15 years as approached governments to disclose why does they create all these secondary water line systems. This would fit right into that narrative of biological warfare being delivered through water systems. Have you seen this before? Not till right now. That's pretty amazing that they've actually revealed that they were doing these water system tests. Our government has been testing on their own personnel without their knowledge. Without their knowledge using toxins from the venom of sea creatures. Hydroxychloroquine. Hydroxychloroquine. It's a derivative of chlorine. Yes, it is. Chlorine dioxide. I don't know if you saw the interview with Mike Adams, but this guy got on there and his name is Bob the Plumber. He has documents that show chlorine dioxide is inhibitory and destroys every aspect of King Cobra venom, even in water. But there is chlorine in the water so then how would this work? Are they just the right amounts? What if the whole water treatment thing with the CDC right now, looking for SARS-CoV-2 in your poop water? What if they're actually controlling how much chlorine is going into your town, reducing it, putting in the toxins to create SARS-CoV-2 outbreaks in your town using venomous peptides like they were doing in the 70s in federal buildings through water delivery systems? And then what if they're actually testing to see if they increase the chlorine now after creating the outbreak? They can actually monitor the chlorine input into your water and how much less SARS-CoV-2 organisms are coming out of that community and they're just managing the outbreak, creating the outbreak, and controlling the subsiding of that outbreak with simple chlorine in the water. They've already been managing chlorine input and output in your drinking water. Going back to your point about how it would be done is that once you've got venom in circulation, venom would be able to be transported the same way as the spinning cobra uses it in the salivary bubble and it would be spat out. It could also just free float in the air because it's charged. So the basic science of Brownian motion is that particles that are charged stay up in the air. You could absolutely add this to the water if somebody chose to do it. The difference maybe between the way Dr. Artis and I were thinking through this is that I do believe that the initial attacks included a viral vector, SARS-CoV-1, which was a bad coronavirus with the spike protein, the new developed spike protein attached to SARS-CoV-1 and now gets called SARS-CoV-2 with an adapted gain of function spike protein. And then your venom then replicates. So the machinery to get this out into the air, into the water, into the food, onto someone's hands, onto someone's clothing, it's the human body that's the manufacturer of that venom. Here's how it works. This is the virus. These are called spike proteins. Scientists have isolated the genetic code for those proteins, a set of instructions that are put into the vaccine so your body knows how to make them. Your immune system won't make an actual virus particle, but it will make copies of the angry red spike protein. Then you don't need the virus anymore because then you start vaccinating people and you're putting the venom and then they're telling the public, they're telling the public that they're going to be making the spike proteins. So if you understand then that the spike proteins have venom on it, then what they've told the public is that for a certain amount of time, if you get an mRNA vaccine, for example, that your body is going to be producing the spike proteins and therefore your human body is making venom that you're shedding to the rest of the world. Dr. Fauci, knowing that it is a crime to lie to Congress, do you wish to retract your statement of May 11th where you claimed that the NIH never funded gain of function research in Wuhan? Senator Paul, I have never lied before the Congress and I do not retract that statement. This paper that you were referring to was judged by qualified staff up and down the chain as not being gain of function. What was, let me finish. You take an animal virus and you increase its responsibility to humans. You're saying that's not gain of function? That is correct. Senator Paul, you do not know what you are talking about, quite frankly, and I want to say that officially. You do not know what you are talking about. Animal viruses that only occur in animals and they increase their transmissibility to humans. How you can say that is not gain of function? It is not. It's a dance and you're dancing around this because you're trying to obscure responsibility for 4 million people dying around the world from a pandemic. And let's send Dr. Fauci. I have to. Now you're getting into something. If the point that you are making is that the grant that was funded as a sub-award from EcoHealth to Wuhan created SARS-CoV-2. That's where you are getting. Let me finish. We don't know. Well, wait a minute. I can share with you. I don't know if it did come to the lab, but all the evidence is pointing that it came from the lab. You... And there will be responsibility for those who funded the lab, including yourself. And you are implying that what we did was responsible for the deaths of individual. I totally resent that. And if anybody is lying here, Senator, it is you. You have a look at this paper I found from 1977 that spells out how they took a coronavirus and they successfully got a leptin, a form of venom, just to call it a coronavirus. And so, you know, that to me is like, I shouldn't be trying to explain that to the public. So someone's been vaccinated. They're concerned about this. They believe it's venom or venom peptides. How do they counter this? How do they get it out? Yeah, this is great. So, the research was done by Kathleen Crico and Drew Weissman. They use this thing called snake venom phosphodiesterase. They say to actually do the gene editing therapy, to cleave RNA and DNA, to get the mRNA inside of it. That's what they've been doing. This snake venom phosphodiesterase, you can actually buy it online at Innovative Research and it actually states on there, it says you cannot mix it with any of these four things. You can't mix it with glutathione. You can't mix it with any cysteine products like inacetylcysteine. You cannot mix it with vitamin C or ascorbic acid and you can't mix it with EDTA. Any of these four things will completely inhibit or destroy this gene editing snake venom phosphodiesterase. I would suggest that if anyone's had the shots, the mRNA shots, and they're having any symptoms ever since, I would absolutely recommend they consult an infusion center for glutathione injections and vitamin C high dose injections. Let that clinic or those clinicians who are experts in that field tell you based on your medical history, weight, size, how often to do it, but if it actually is known to negate and inhibit the mechanism of action of that substance found in snake venom, that we know is being used since 2009 as the source to do cleaving of your DNA by the researchers who created these shots, that's number one what I would do. I would recommend everybody get on high dose vitamin C orally. If you've been injured by the shots, please get it intravenously and then and glutathione also. With the nicotine and melatonin? Oh yeah, I would definitely try those. Yep, I would try the nicotine probably two to three days at most and see if you see improvements. Melatonin I would just stay on. These scientists have been grappling with how to stop sepsis, one of the most deadly complications of COVID-19. It was very critically ill with multiple organ failure and we were fearful that he wasn't going to make it. They administered a dose of vitamin C 50 times greater than what's been tried before. For a human if you got this dose it would be something like 5000 oranges worth of vitamin C. The response was one of the most amazing things I've seen in my research career. Right now if you search vitamin C on the internet, search results analyzing its potential impact on the coronavirus show up immediately. A doctor in New York said he's seen promising response in patients receiving upwards of 6000 milligrams per day. One of the most exciting aspects of what we're seeing with vitamin C is the amount of time that it's able to reduce ventilator times for patients that are in the hospital. In China they're already proving the success rate with 10,000 to 20,000 milligrams a day for mild to moderate cases and Dr. Mao reported that they had a patient declining in front of them. They gave him 50,000 milligrams in four hours and they watched all of his respiratory sats and everything improve in front of their eyes. I just want to continue to propose it's going to be what you take care of internally. I've already said this is a massive eugenics program, a massive genocide program. They are using biological weapons that I believe are being targeted and used and extracted and isolated and synthetically made venoms from shellfish toxin, snake venom toxins and that's how they're creating this entire pandemic. And they're using those same biological weapons inside the vaccines, which is why you're seeing such horrible detrimental effects. And what we have heard consistently is zinc load, vitamin C load, D, glutathione, NAC, these are all going to be very protective internally against any deadly disease processes even of biological weapons. The vaccines are not the same as the actual gain of function weapon they used from the beginning. They're different and what would make it different is what allows any cell in your body to be gene edited. And that's what Anthony Fauci and the NIH have been funding for at least 13 years now with these two people. Is this the reverse transcriptase? Yeah, they use reverse transcriptase to actually get this mRNA inside your cell and they're telling your cells they want your cells to now generate a spike protein all on its own, which is similar to the spike protein on SARS-CoV-2. The problem with that is the Salk Institute actually said that the spike protein without the virus all by itself destroys 28 different tissues in your body and they want your body to make it. So now everyone around the world either gets to blindly ignore the warning and dismiss it outright or do like a whole bunch of people around the world are doing right now, which is phenomenal. They are doing their own research and they're trying to figure out the mystery because for the first time for many of them, the insanity, the disgusting amount of injuries from these vaccines, the mysteries of the side effects and symptoms of COVID-19 finally makes sense to many as they sit and look at this. They're like, oh my God, it all makes sense. Yes, it does. Once you look at it for a minute, you'll start to see the reality that I see. This makes it tempting to ignore the wise restraints that make men free. But each time we do so, each time the means we use are wrong, our inner strength, the strength which makes us free, is lessened. The difference of this forced industrial revolution is it doesn't change what you are doing. It changes you if you take a genetic editing just as an example. It's you who are changed and of course it has a big impact on your identity. The goal is to destroy all existing religions, save theirs, all existing governments, save theirs and shackle the mob in a system of eternal oppressive debt changed to a computer for the rest of their life and a propagandized rule to make them believe that they are happy in this system. And then the Blessed Mother is always every statue, she's standing on the head of a serpent. You know, you've heard the term symbolism will be their downfall. Well then you think about the snake in the Garden of Eden and then you see the cover of Time magazine 1999 where they show human DNA and they show a snake around it in the human genome. When you began to do that kind of gene editing, some people worried that you were changing what it means to be human. That's the problem. Of course, the new industrial revolution offers us many opportunities but it raises many full questions on the ethical but even legal implications. In conclusion, and despite all the challenges, we have to uphold our responsibility which we have towards the next generation and which we can only fulfill through collaboration on a national and on a global level. It is basically biological chip that it is in the tablet and once you take the tablet and dissolve it into your stomach, it sends a signal that you took the tablet. So imagine the applications of that compliance.
"""

In [5]:
extracted_text

"\nLadies and gentlemen, the world has to overcome not only the damage done to our economies and our societies by COVID-19, it also has to confront the repercussions of a dangerous clash between major global powers. History is truly at a turning point. Get back in the quest to understand coronavirus after a researcher is shot and killed. I have decided that the United States of America will renounce the use of any form of deadly biological weapons. Get your shots, get your boosters. Doctors baffled by increase in sudden adult death syndrome. We destroyed all of our seed stocks. We destroyed all of our production material at Pinebuff, Arkansas, and we completely got out of the biological warfare business. Subject today concerns CIA's involvement in the development of bacteriological warfare materials with the Army's Biological Laboratory of Fort Detrick. In the course of the investigation, CIA's laboratory storage facilities were searched and about 11 grams, a little less than half an o

In [7]:
# Load your usual SpaCy model (one of SpaCy English models)
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40155833/40155833 [00:30<00:00, 1328377.49B/s]


In [8]:
doc = nlp(extracted_text)
graph_text = doc._.coref_resolved

In [25]:
def load_model():
    print("Loading model...")
    tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
    model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
    print("Model loaded!")
    return tokenizer, model

tokenizer, model = load_model()

Loading model...


Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Model loaded!


In [43]:
def from_text_to_kb(text, model, tokenizer, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                article_url: {
                    "spans": [spans_boundaries[current_span_index]]
                }
            }
            kb.add_relation(relation, article_title, article_publish_date)
        i += 1

    return kb


def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations


def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article

def from_url_to_kb(url, model, tokenizer):
    article = get_article(url)
    config = {
        "article_title": article.title,
        "article_publish_date": article.publish_date
    }
    kb = from_text_to_kb(article.text, model, tokenizer, article.url, **config)
    return kb

def get_news_links(query, lang="en", region="US", pages=1):
    googlenews = GoogleNews(lang=lang, region=region)
    googlenews.search(query)
    all_urls = []
    for page in range(pages):
        googlenews.get_page(page)
        all_urls += googlenews.get_links()
    return list(set(all_urls))

def from_urls_to_kb(urls, model, tokenizer, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(urls)} links to visit")
    for url in urls:
        if verbose:
            print(f"Visiting {url}...")
        try:
            kb_url = from_url_to_kb(url, model, tokenizer)
            kb.merge_with_kb(kb_url)
        except ArticleException:
            if verbose:
                print(f"  Couldn't download article at url {url}")
    return kb

def save_network_html(kb, filename="network.html"):
    # create network
    net = Network(directed=True, width="700px", height="700px")

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    # edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"],
                    title=r["type"], label=r["type"])

    # save network
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')
    net.show(filename)

def save_kb(kb, filename):
    with open(filename, "wb") as f:
        pickle.dump(kb, f)

class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if name == 'KB':
            return KB
        return super().find_class(module, name)

def load_kb(filename):
    res = None
    with open(filename, "rb") as f:
        res = CustomUnpickler(f).load()
    return res


In [44]:
kb = from_text_to_kb(graph_text, model, tokenizer, "", verbose=True)

Input has 13423 tokens
Input has 105 spans
Span boundaries are [[0, 128], [127, 255], [254, 382], [381, 509], [508, 636], [635, 763], [762, 890], [889, 1017], [1016, 1144], [1143, 1271], [1270, 1398], [1397, 1525], [1524, 1652], [1651, 1779], [1778, 1906], [1905, 2033], [2032, 2160], [2159, 2287], [2286, 2414], [2413, 2541], [2540, 2668], [2667, 2795], [2794, 2922], [2921, 3049], [3048, 3176], [3175, 3303], [3302, 3430], [3429, 3557], [3556, 3684], [3683, 3811], [3810, 3938], [3937, 4065], [4064, 4192], [4191, 4319], [4318, 4446], [4445, 4573], [4572, 4700], [4699, 4827], [4826, 4954], [4953, 5081], [5080, 5208], [5207, 5335], [5334, 5462], [5461, 5589], [5588, 5716], [5715, 5843], [5842, 5970], [5969, 6097], [6096, 6224], [6223, 6351], [6350, 6478], [6477, 6605], [6604, 6732], [6731, 6859], [6858, 6986], [6985, 7113], [7112, 7240], [7239, 7367], [7366, 7494], [7493, 7621], [7620, 7748], [7747, 7875], [7874, 8002], [8001, 8129], [8128, 8256], [8255, 8383], [8382, 8510], [8509, 8637], [

/home/titan/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/titan/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [47]:
kb.relations

[{'head': 'Coronavirus',
  'type': 'has effect',
  'tail': 'COVID-19',
  'meta': {'': {'spans': [[0, 128], [8636, 8764]]}}},
 {'head': 'United States',
  'type': 'instance of',
  'tail': 'Society',
  'meta': {'': {'spans': [[0, 128]]}}},
 {'head': 'Assassination',
  'type': 'subclass of',
  'tail': 'Mass killing',
  'meta': {'': {'spans': [[254, 382]]}}},
 {'head': 'Kidney failure',
  'type': 'has effect',
  'tail': 'Heart failure',
  'meta': {'': {'spans': [[635, 763]]}}},
 {'head': 'Heart failure',
  'type': 'has cause',
  'tail': 'Kidney failure',
  'meta': {'': {'spans': [[635, 763]]}}},
 {'head': 'Liver failure',
  'type': 'has effect',
  'tail': 'Heart failure',
  'meta': {'': {'spans': [[635, 763]]}}},
 {'head': 'COVID-19',
  'type': 'drug used for treatment',
  'tail': 'Remdesivir',
  'meta': {'': {'spans': [[762, 890]]}}},
 {'head': 'Remdesivir',
  'type': 'medical condition treated',
  'tail': 'COVID-19',
  'meta': {'': {'spans': [[762, 890]]}}},
 {'head': 'COVID-19',
  'type

In [49]:
save_network_html(kb, filename="networks/network.html")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
